In [1]:
import tensorflow as tf
from tensorflow import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

2024-11-01 22:56:22.012448: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 22:56:22.221413: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-01 22:56:23.257248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 22:56:23.257342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 22:56:23.305562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
model = keras.models.load_model('model-ni_ppmi_vgg16.h5')

2024-11-01 22:56:27.847356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-01 22:56:27.848442: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
from keras.preprocessing.image import load_img
from keras.applications.resnet import preprocess_input
import numpy as np

In [9]:
img = load_img('./control_raw_brain_mri_w_sn_1.jpg', target_size=(512, 512))

In [10]:
x = np.array(img)
X = np.array([x])
X.shape

(1, 512, 512, 3)

In [11]:
X = preprocess_input(X)
pred = model.predict(X)
pred

1/1 [==============================] - 0s 198ms/step


array([[0.13504983, 0.880159  ]], dtype=float32)

In [12]:
pred = {"prediction": pred[0][1]}
pred

{'prediction': 0.880159}

# Convert Keras to TF-Lite

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model_ni_ppmi_vgg16.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


2024-11-01 23:02:10.687809: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-11-01 23:02:10.687869: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-11-01 23:02:10.688084: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmppt3r8csr
2024-11-01 23:02:10.690755: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-11-01 23:02:10.690789: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmppt3r8csr
2024-11-01 23:02:10.697093: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-11-01 23:02:10.773786: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmppt3r8csr
2024-11-01 23:02:10.792567: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 104481 

In [15]:
import tensorflow.lite as tflite

In [17]:
interpreter = tflite.Interpreter(model_path='model_ni_ppmi_vgg16.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [19]:
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.8801586}

# Removing TF dependency

In [22]:
from PIL import Image

In [29]:
with Image.open('./control_raw_brain_mri_w_sn_1.jpg') as img:
    img = img.resize((512, 512), Image.NEAREST)

In [30]:
def preprocess_input(x):
    x = x[..., ::-1]
    mean = [103.939, 116.779, 123.68]
    x[..., 0] -= mean[0]
    x[..., 1] -= mean[1]
    x[..., 2] -= mean[2]
    return x

In [31]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [32]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.8801586}

In [23]:
!pip install keras-image-helper

In [24]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 423.9 kB/s eta 0:00:0000:0100:01


In [33]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [34]:
interpreter = tflite.Interpreter(model_path='model_ni_ppmi_vgg16.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [38]:
preprocessor = create_preprocessor('vgg16', target_size=(512, 512))

In [41]:
url = 'images/test images/control_raw_brain_mri_w_sn_1.jpg?raw=true'
X = preprocessor.from_url(url)

In [42]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.0023406097}